In [1]:
import pandas as pd
import re

# 1. Load CSV Data
orders_df = pd.read_csv('orders.csv')

# 2. Load JSON Data
users_df = pd.read_json('users.json')

# 3. Load SQL Data (Parsing INSERT statements)
restaurant_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith("INSERT INTO restaurants VALUES"):
            # Extract content inside parentheses: (id, 'name', 'cuisine', rating)
            match = re.search(r"\((.*)\);", line)
            if match:
                content = match.group(1)
                # Split by comma (assuming simple format from snippet)
                parts = [p.strip().strip("'") for p in content.split(',')]
                restaurant_data.append(parts)

restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
restaurants_df['rating'] = pd.to_numeric(restaurants_df['rating'])

# 4. Merge the Data
# Merge orders with users
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')
# Merge with restaurants
final_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))

# Rename 'name' to 'user_name' for clarity
final_df.rename(columns={'name': 'user_name'}, inplace=True)

# 5. Save Final Dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)